In [2]:
from getTexts import *
import pandas as pd 
import re 

In [125]:
csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/Early-Modern-London/Relevant_Metadata/charityTCP.csv')
tcpIDs = [ _ for _ in csv_data['id']]

In [ ]:
'''
Extract the entire body text from an XML document.  
'''
outputfolder = 'charityTXT'
convert(tcpIDs,outputfolder)
print(underscores)
convert(underscores,outputfolder)

'''
Processing 190 texts takes over 15 minutes 

TCP charity texts that are not in EP: 
'B33867', 'B13871', 'B13868', 'B13878', 'B13862', 'B13860', 
'B09696', 'A86972', 'A19579', 'A09957'
'''

In [152]:
# patterns = ["Preach't at (.*?)[^\w\s\-]",'Preached at (.*?)[^\w\s\-]','preached at (.*?)[^\w\s\-]','preached in (.*?)[^\w\s\-\']']
patterns1 = ["preach't (.*?)$","preached at (.*?)$","preached in (.*?)$","preacht (.*?)$","before the (.*?)$", "before his (.*?)$"]
patterns2 = ["preacher at (.*?)$","preacher in (.*?)$","pastor at (.*?)$","pastor of (.*?)$"]
places = []
pastorPlaces = []
for title in csv_data['title']:
    foundPattern1 = False
    for pattern in patterns1: 
        place = re.search(rf'{pattern}', title)
        if place: 
            places.append(place.group())
            foundPattern1 = True
    if not foundPattern1: 
        for pattern in patterns2: 
            place = re.search(rf'{pattern}', title)
            if place: 
                pastorPlaces.append(place.group())
                foundPattern2 = True

print(len(places), len(pastorPlaces), len(places)+len(pastorPlaces))
print(places)
print(pastorPlaces)

38 27 65
['preached in the Abbey Church of Westminster before the house of Peers, on the 24th of September, 1645. being the day of their monethly fast. / By William Gouge, one of the members of the Assembly.', 'before the house of Peers, on the 24th of September, 1645. being the day of their monethly fast. / By William Gouge, one of the members of the Assembly.', 'before his majestie at his court of Thebalds, on Sunday, Sept. 15. 1622 In the ordinary course of attendance. By Ios. Hall D.D.', 'before the Honourable House of Commons assembled in Parliament. At a publick fast, 29. Iune, 1642. By William Gouge.', 'before the Lords on a day of humiliation for a blessing on a treaty between His Majesties and the Parliaments commissioners. / By W. Gouge.', "preach't to his Maiesty, at the court of White-hall. Aug.8. / By Jos. B. of Exon.", 'preached at Paules Crosse, vpon the fourth of December, out of Luke. 12. 15. By William Whatelie, preacher of the word of God, in Banbury', 'preached at t

In [69]:
def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [ ]:
'''
For efficiency, extract notes (marginalia) directly from TCP (parsing EP files takes much longer)
'''
folder = f'/Users/amycweng/Digital Humanities/charityTXT'
for tcpID in tcpIDs: 
    path = findTextTCP(tcpID)
    with open(path,'r') as file: 
        data = file.read()
    tag = SoupStrainer("note")
    soup = BeautifulSoup(data,parse_only=tag,features='html.parser')
    note_text = notes(soup)
    if note_text != '': 
        with open(f'{folder}/{tcpID}NOTES.txt') as file: 
            file.write()

In [90]:
'''
Extracting title pages directly from TCP. 
Title pages often contain information about where the sermon was preached. 
'''
import csv 
folder = f'/Users/amycweng/Digital Humanities/'
outfile = open(f'{folder}/sermon_titles.csv','a+')
columns = ['id','title page']
writer = csv.DictWriter(outfile, fieldnames=columns)
writer.writeheader()

for tcpID in tcpIDs: 
    path = findTextTCP(tcpID)
    with open(path,'r') as file: 
        data = file.read()
    tag = SoupStrainer("div1", attrs={'type':'title page'})
    soup = BeautifulSoup(data,parse_only=tag,features='html.parser')
    title = []
    sentences = soup.find_all('p')
    for sentence in sentences: 
        title.append(sentence.text)
    if len(title) != 0: 
        title = ' '.join(title)
        title = re.sub('\n',' ',title)
        t = re.search(r'^(.*?) \d{4}',title)
        if t: 
            writer.writerow({'id': tcpID,  'title page': t.group()})
        else: 
            writer.writerow({'id': tcpID,  'title page': title})
outfile.close()